# Загрузка данных

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('./data/marketing_AB.csv', index_col=[0])

# Исследование данных

In [14]:
# 1. Исследование структуры данных
print("Информация о датасете:")
print(df.info())  # Общая информация о столбцах


Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB
None


In [29]:
print("\nПример данных:")
df.sample(15)# примеры данных


Пример данных:


,user id,test group,converted,total ads,most ads day,most ads hour
253955,1554625,ad,False,17,Monday,15
430739,1467393,ad,False,7,Saturday,15
306783,912258,psa,False,14,Sunday,19
295708,1209115,ad,False,5,Friday,14
531771,1007023,ad,False,1,Saturday,10
252745,1634772,ad,False,32,Monday,11
227222,1224341,ad,False,9,Thursday,19
2465,1445877,ad,False,13,Tuesday,22
423498,906059,psa,False,11,Friday,22
513017,1517240,ad,False,7,Wednesday,13


In [13]:
# Разделяем на контрольную и тестовую группы
control_group = df[df["test group"] == "psa"]
test_group = df[df["test group"] == "ad"]

# Рассчитываем метрики
metrics = {
    "control_group": {
        "visits": len(control_group),  # Количество посещений
        "total_conversions": control_group["converted"].sum()  # Суммарное количество покупок
    },
    "test_group": {
        "visits": len(test_group),  # Количество посещений
        "total_conversions": test_group["converted"].sum()  # Суммарное количество покупок
    }
}

# Вывод результатов
print("Результаты анализа:")
print(metrics)

Результаты анализа:
{'control_group': {'visits': 23524, 'total_conversions': 420}, 'test_group': {'visits': 564577, 'total_conversions': 14423}}


In [41]:
sorted(df['user id'].value_counts())

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [16]:
# 2. Преобразование столбцов к необходимым типам данных
# Преобразование столбца converted в тип bool (если это строка или другие значения)
if df["converted"].dtype != "bool":
    df["converted"] = df["converted"].astype(bool)

# Преобразование total_ads в целые числа
if df["total ads"].dtype != "int64":
    df["total ads"] = df["total ads"].astype(int)



In [ ]:
# 3. Проверка на наличие пропусков
print("\nКоличество пропусков по столбцам:")
print(df.isnull().sum())

# Удаление строк с пропусками не требуется ввиду их отсутствия


Количество пропусков по столбцам:
user id          0
test group       0
converted        0
total ads        0
most ads day     0
most ads hour    0
dtype: int64


In [ ]:
# 4. Проверка на пересечение пользователей между группами
# Создаем два поднабора данных: контрольная группа и тестовая группа
control_group = df[df["test group"] == "psa"]
test_group = df[df["test group"] == "ad"]
print(f'Размер тестовой выборки = {test_group.shape[0]}, Размер котрольной выборки = {control_group.shape[0]}')

Размер тестовой выборки =564577, Размер котрольной выборки = 23524


In [ ]:
df_int = pd.merge(test_group, control_group, how ='inner', on =["user id"])

In [38]:
len(df_1)

0

In [24]:

# Ищем пользователей, которые находятся в обеих группах
overlap_users = set(control_group["user id"]).intersection(set(test_group["user id"]))
print("\nКоличество пользователей, попавших в обе группы:", len(overlap_users))


Количество пользователей, попавших в обе группы: 0


In [19]:

# Исключаем пользователей, попавших в обе группы
df = df[~df["user id"].isin(overlap_users)]

# Проверяем, что пересечения больше нет
print("\nПроверка пересечений после удаления:")
control_group = df[df["test group"] == "psa"]
test_group = df[df["test group"] == "ad"]
overlap_users = set(control_group["user id"]).intersection(set(test_group["user id"]))
print("Количество пользователей, попавших в обе группы после очистки:", len(overlap_users))


Проверка пересечений после удаления:
Количество пользователей, попавших в обе группы после очистки: 0
